<a href="https://colab.research.google.com/github/bt86548/pytorch/blob/master/car_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [ ]:

REBUILD_DATA = True

class car_test():
    IMG_SIZE = 80 #把所有圖片尺寸都統一,可能會失真需要先測試一下圖片
    toyota_camry = "/content/drive/My Drive/iii_course/toyota_test/camry" 
    toyota_vios = "/content/drive/My Drive/iii_course/HOT/HOT_image/toyota/vios"
    toyota_altis="/content/drive/My Drive/iii_course/toyota_test/altis"

    # TESTING = "PetImages/Testing"
    LABELS = {toyota_camry: 0, toyota_vios: 1,toyota_altis:2} #貼標籤
    training_data = []

    
    toyota_camry_count = 0 #計數很重要 需要知道做完沒
    toyota_vios_count = 0
    toyota_altis_count = 0


    def make_training_data(self):
        for label in self.LABELS: #用標籤去跑所有的圖像
            for f in tqdm(os.listdir(label)): #跑遍所有圖片,tqdm進度條
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f) #label為/content/PetImages2/PetImages/Cat位置,f為每張照片,以此類推去讀取
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) #把讀到的照片變成灰階
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE)) #把每張照片都變50*50的大小
                        self.training_data.append([np.array(img), np.eye(3)[self.LABELS[label]]])  
                        #np.array(img)為該圖片的矩陣,np.eye(2)[self.LABELS[label]]]
                        # do something like print(np.eye(2)[1]), just makes one_hot  np.eye(2)為對角線為1的2X2矩陣
                        #print(np.eye(2)[self.LABELS[label]]) 

                        if label == self.toyota_camry:
                            self.toyota_camry_count += 1
                        elif label == self.toyota_vios:
                            self.toyota_vios_count += 1
                        elif label == self.toyota_altis:
                            self.toyota_altis_count += 1

                    except Exception as e:
                        pass
                        #print(label, f, str(e))

        np.random.shuffle(self.training_data) #重新洗牌讓他更準
        np.save("training_data.npy", self.training_data) #存成npy檔讓圖片比較好辨識
        print('toyota_camry:',car_test.toyota_camry_count)
        print('toyota_vios:',car_test.toyota_vios_count)
        print('toyota_altis:',car_test.toyota_altis_count)

if REBUILD_DATA:
    car_test = car_test()
    car_test.make_training_data()

FileNotFoundError: ignored

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)

        x = torch.randn(80,80).view(-1,1,80,80)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)

    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)



if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

net = Net().to(device)

In [ ]:
training_data = np.load("/content/drive/My Drive/training_data (1).npy", allow_pickle=True)
print(len(training_data))

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

In [ ]:
X = torch.Tensor([i[0] for i in training_data]).view(-1,80,80)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)
print(val_size)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X))
print(len(test_X))

In [ ]:
def train(net):
    BATCH_SIZE = 50 #batch size 不能太大 不然cuda會爆掉
    #如果Batch_Size過小，訓練資料就會非常難收斂，從而導致underfitting。
    # 增大Batch_Size,相對處理速度加快。
    # 增大Batch_Size,所需記憶體容量增加（epoch的次數需要增加以達到最好的結果)
    EPOCHS = 3
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
            batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,80,80)
            batch_y = train_y[i:i+BATCH_SIZE]
  

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            net.zero_grad()
            
            outputs = net(batch_X)
            print('output:',outputs)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()
        print('===========================================')
        print('loss:',loss)

train(net)

In [ ]:
def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i]).to(device)
            

            net_out = net(test_X[i].view(-1, 1,80,80).to(device))[0]

            predicted_class = torch.argmax(net_out)
            if predicted_class == real_class:
                correct += 1
            total += 1
    print("Accuracy:", round(correct/total,3))

test(net)

In [ ]:
def batch_test(net):
    BATCH_SIZE = 100
    correct = 0
    total = 0
    with torch.no_grad():
        #np.random.shuffle(test_X)
        #np.random.shuffle(test_y)

        batch_X = test_X[:BATCH_SIZE].view(-1,1,80,80)
        batch_y = test_y[:BATCH_SIZE]

        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        net.zero_grad()
        outputs = net(batch_X)

        matches  = [torch.argmax(i)==torch.argmax(j) for i, j in zip(outputs, batch_y)]
        acc = matches.count(True)/len(matches)

        print("Test Accuracy:", round(acc, 3))


batch_test(net)

In [ ]:
def fwd_pass(X, y, train=False):

    if train:
        net.zero_grad()
    outputs = net(X)
    matches  = [torch.argmax(i)==torch.argmax(j) for i, j in zip(outputs, y)]
    acc = matches.count(True)/len(matches)
    loss = loss_function(outputs, y)

    if train:
        loss.backward()
        optimizer.step()

    return acc, loss

In [ ]:
net = Net().to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

def train(net):
    BATCH_SIZE = 100
    EPOCHS = 3
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
            batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,80,80)
            batch_y = train_y[i:i+BATCH_SIZE]

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            acc, loss = fwd_pass(batch_X, batch_y, train=True)

            print(f"Acc: {round(float(acc),2)}  Loss: {round(float(loss),4)}")

            # just to show the above working, and then get out:
            if i == 5:
                break
            break

train(net)